# Исследование надёжности заёмщиков

**Задача:**

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроем файл с данными и изучим общую информацию.

In [1]:
#Загрузим статистику от банка 
import pandas as pd
#import seaborn as sns

df = pd.read_csv('C:\Python\_File_test\data.csv')
# Предпосмотр таблицы

df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
# <получение общей информации о данных в таблице df>
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 колонок с различными типами данных 
    
Подробно разберём, какие в df столбцы и какую информацию они содержат:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

Количество значений в столбцах различается. Это говорит о том, что в данных есть <пустые> значения.

**Выводы**

Каждая строка таблицы содержит информацию о клиенте - количество детей, стаж, возраст, образование, семейное положение, доход, пол, тип занятости и т.д. Для проверки гипотезы особенно ценны столбцы *debt*, *children*, *family_status*. 
Проблемы, которые нужно решать: 
* пропуски в колонках *days_employed* и *total_income* и ошибки данных;
* столбец *children* содержит "подозрительные" значения <-1>, <20>;
* *dob_years* имеет значение <0>;
* *education* проблемы с верхним и нижним регистрами, необходимо привести к однообразию данные;
* *education_id* имеет пропуски, их можно устранить из данных колонки *education* - пропуски были в дубликатах, данные строки удалены;
* *gender* присутствует значение < XNA >.


## Предобработка данных

**- 1 -**

In [3]:
# Проанализируем таблицу на наличие пустых данных
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Присутствуют пустые данные в колонках :
* days_employed
* total_income

In [4]:
# проверка доли пустых данных от общего количества
days_employed_na_part = df.isna().total_income.sum() / df.shape[0]
print('Доля пропущенных значений на примере колонке <total_income> составляет {:.2%}'.format(days_employed_na_part))

Доля пропущенных значений на примере колонке <total_income> составляет 10.10%


In [5]:
# проверим есть ли данные в колонке total_income по тем строкам где days_employed пустые
df.isnull().query('days_employed == True').agg(['sum']).loc[:, ['days_employed','total_income']]

,days_employed,total_income
sum,2174,2174


In [6]:
# вычислим медиану total_income и сохраним в переменную total_income_median
total_income_median = df.total_income.median()
#  заменить пустые ячейки медианным значением колонки total_income
df.total_income = df.total_income.fillna(total_income_median)
# проверим, удала ли команда данные, заменив их медианным значением. если результат "0" - все получилось. 
df.total_income.isna().sum()

0

**Наблюдения по пустым данным:**
* все строки где в колонке days_employed значения пустые, соответсвуют пустым данным в total_income;
* доля пропущенных значений составляет **10.10%** от общего количества данных что может повлиять на результаты исследования;
* пропуски данных, возможно, связаны с некорректной работой автоматизированным сохраннеием данным, необходимо сообщить разработчикам для проверки;
* пропущенные значения в колонке total_income заполнены медианным значением колонки т.к в колонке total_income присутствуют данные сильно выделяющиеся.

**- 2 -**

В колонке < days_employed > присутствуют пропуски, а так же аномалии в виде отрицательноых значений у работающих сотрудников. 
Вероятно ошибка кроется в формуле расчета стажа : если не указана дата выхода на пенсию он вычитает количество стажа из 0, хотя следует вычитать от текущей даты. **(Необходимо уводомить разработчиков)** Отричательные данные можно заменить модульным значением и это не повлечет ошибки. Пропуски можно заменить медианным значением. 

In [7]:
# функция для замены отрицательных значний абсолютным
def abs_if_negative(x):
    if x < 0:
        return abs(x)
    return x 


In [8]:
# применим функцию abs_if_negative() к колонке days_employed для замены отрицательных значений модульными
df.days_employed = df.days_employed.apply(abs_if_negative)

In [9]:
# вычислим медиану days_employed и сохраним в переменную days_employed_median
days_employed_median = df.days_employed.median()
#  заменить пустые ячейки медианным значением колонки days_employed
df.days_employed = df.days_employed.fillna(days_employed_median)
# проверим, удала ли команда данные, заменив их медианным значением. если результат "0" - все получилось. 
df.days_employed.isna().sum()

0

Колонка < children > содержит "подозрительные" значения <-1>, <20>;
* под значением <-1> вероятно подразумевалось значение <0> или <1> => сделаем предположение что корректное значение <1>
* под значением <20> может быть <2>, <20> или максимальное из остальных значений в колонке - <5> => сделаем предположение что корректное значение <2>

In [10]:
#создадим функцию для замены данных в колонке <children> значений <-1>, <20> соответственно на <1>, <2>
def children_change_value(x):
    if x == -1:
        return 1
    elif x == 20:
        return 2
    else:
        return x


In [11]:
# применим функцию children_change_value() к колонке <children> для замены значений <-1>, <20> соответственно на <1>, <2>
df.children = df.children.apply(children_change_value)
# проверим корректность выполнения функции, если уникальные значения будут в границе [0,5] то функция выполнена успешно
df.children.value_counts() 

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**- 3 -**

In [12]:
# поменяем тип данных колокни <total_income> на целочисленный
df.total_income = df.total_income.astype('int64')

**- 4 -**

In [13]:
# проверим таблица на наличие явных дубликатов - их 54 шт. 
df.duplicated().sum()

54

In [14]:
# удалим явные дубликаты и сбросим индексирование. 
df = df.drop_duplicates().reset_index(drop=True)
# проверим удалиение дубликатов - 0 - значит дубликаты удалены 
df.duplicated().sum()

0

In [15]:
# проверим какие никальные значения присутствуют в колонке <education> с привязкой по колонке <education_id>
df[['education_id','education']].value_counts(sort=False)

education_id  education          
0             ВЫСШЕЕ                   273
              Высшее                   268
              высшее                  4710
1             СРЕДНЕЕ                  772
              Среднее                  711
              среднее                13705
2             НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
              Неоконченное высшее       47
              неоконченное высшее      668
3             НАЧАЛЬНОЕ                 17
              Начальное                 15
              начальное                250
4             УЧЕНАЯ СТЕПЕНЬ             1
              Ученая степень             1
              ученая степень             4
dtype: int64

В колонке < education > одни и те же значения записаны по разному (разный регистр). Причиной появления не явных дубликотов в колонке вероятно вызвано ручным методом ввода данных, данную проблему можно избежать настроив всплывающее окно с выриантами.

Можно отредантировать данные используя колонке < education_id > применив логику категоризацию : 

< education_id > - < education > 
* 0 - 'высшее'
* 1 - 'среднее'
* 2 - 'неоконченное высшее'
* 3 - 'начальное'
* 4 - 'ученая степень'

In [16]:
# создадим функию для замены названий категоризации колонки образования. 
def education_change_rank(row):
    education_rank_id = row['education_id']
    
    if education_rank_id == 0:
        return 'высшее'
    elif education_rank_id == 1:
        return 'среднее'
    elif education_rank_id == 2:
        return 'неоконченное'
    elif education_rank_id == 3:
        return 'начальное'
    elif education_rank_id == 4:
        return 'ученая степень'
    

In [17]:
# применим функицю замены названий категоризации колонки образования и выведем на экран количество уникальных значений 
# колонки education для проверки результата работы функции - должно остатся 6 вариантов.
df.education = df.apply(education_change_rank, axis=1)
df[['education_id','education']].value_counts(sort=False)

education_id  education     
0             высшее             5251
1             среднее           15188
2             неоконченное        744
3             начальное           282
4             ученая степень        6
dtype: int64

**- 5 -**

Создадимдва новых датафрейма, в которых:

   * < df_education_rank> - каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
   * <df_family_status> - каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.


In [18]:
# создадим датафрейм df_education_rank
df_education_rank = df[['education_id','education']].drop_duplicates().sort_values('education_id').reset_index(drop=True)
df_education_rank

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное
3,3,начальное
4,4,ученая степень


In [19]:
# создадим датафрейм df_family_status
df_family_status = df[['family_status_id','family_status']].drop_duplicates().sort_values('family_status_id').reset_index(drop=True)
df_family_status

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [20]:
# удалим колонки <family_status> и <education> из исходного датафрейма
df = df.drop(['education','family_status'], axis='columns')
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


**- 6 -**

Создадим столбец < total_income_category > с категориями в зависимости от значения total_income:

* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

In [21]:
# создадим функцию разбивки по требуемым категориям 
def total_income_rank(row):
    total_income_value = row['total_income']
    if 0 < total_income_value <= 30000:
        return 'E | 0–30000 |'
    elif total_income_value <= 50000:
        return 'D | 30001–50000 |'
    elif total_income_value <= 200000:
        return 'C | 50001–200000 |'
    elif total_income_value <= 1000000:
        return 'B | 200001–1000000 |'
    elif total_income_value > 1000000:
        return 'A | 1000001 и выше |'

In [22]:
# создадим колонку <total_income_category> применив функцию total_income_rank
df['total_income_category'] = df.apply(total_income_rank, axis=1)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B | 200001–1000000 |
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C | 50001–200000 |
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C | 50001–200000 |
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B | 200001–1000000 |
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C | 50001–200000 |


**- 7 -**

Создадим колонку < purpose_category >, в которую войдут категории на основе данных колонки < purpose > :
* 'операции с автомобилем',
* 'операции с недвижимостью',
* 'проведение свадьбы',
* 'получение образования'

In [23]:
# создадим функцию для проверки комментариев колонки <purpose> и разбивки по категориям назначения кредита
def check_purpose(purpose_str):
    rank_1 = ['автомоб','машин'] # условия проверки для категории *операции с автомобилем*
    rank_2 = ['жиль', 'недвижимост'] # условия проверки для категории *операции с недвижимостью*
    rank_3 = ['свадьб'] # условия проверки для категории *проведение свадьбы*
    rank_4 = ['образовани'] # условия проверки для категории *получение образования*
    
    for str_value in rank_1: # проверка для 1 категории 
        if  str_value in purpose_str:
            return 'операции с автомобилем'
    for str_value in rank_2: # проверка для 2 категории 
        if  str_value in purpose_str:
            return 'операции с недвижимостью'
    for str_value in rank_3: # проверка для 3 категории 
        if  str_value in purpose_str:
            return 'проведение свадьбы'        
    for str_value in rank_4: # проверка для 4 категории 
        if  str_value in purpose_str:
            return 'получение образования' 

In [24]:
# создадим колонку <purpose_category> используя функцию check_purpose
df['purpose_category'] = df.purpose.apply(check_purpose)
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B | 200001–1000000 |,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C | 50001–200000 |,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C | 50001–200000 |,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B | 200001–1000000 |,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C | 50001–200000 |,проведение свадьбы


In [31]:
# -1-
#создадим сводную таблицу pivot_children_dept для анализа зависимости между количеством детей и возвратом кредита в срок
pivot_children_dept = df.pivot_table(index=['children'], values='debt', aggfunc=['count','sum'])
# переименуем колонки сводной таблицы
pivot_children_dept.columns = ['_'.join(a) for a in pivot_children_dept.columns]
pivot_children_dept = pivot_children_dept.reset_index()\
.rename(columns={'children':'children_count','count_debt':'total_clients', 'sum_debt':'total_dept'})
# для анализа добавим колонку total_percent - отношение количества возвратов в срок к общему количеству клиентов в гуппе
pivot_children_dept['total_percent'] = round((pivot_children_dept.total_clients-pivot_children_dept.total_dept) \
                                             / pivot_children_dept.total_clients * 100, 2)
pivot_children_dept

,children_count,total_clients,total_dept,total_percent
0,0,14107,1063,92.46
1,1,4856,445,90.84
2,2,2128,202,90.51
3,3,330,27,91.82
4,4,41,4,90.24
5,5,9,0,100.00


In [26]:
# -2-
#создадим сводную таблицу pivot_family_dept для анализа зависимости между семейным положением и возвратом кредита в срок
pivot_family_dept = df.pivot_table(index=['family_status_id'], values='debt', aggfunc=['count','sum'])
# переименуем колонки сводной таблицы
pivot_family_dept.columns = ['_'.join(a) for a in pivot_family_dept.columns]
pivot_family_dept = pivot_family_dept.reset_index()\
.rename(columns={'count_debt':'total_clients', 'sum_debt':'total_dept'})
# для удобства добавим в сводную таблицу информацию по статусу семьи в колонку family_status из датасета df_family_status
pivot_family_dept = df_family_status.merge(pivot_family_dept, on='family_status_id', how='left')
# для анализа добавим колонку total_percent - отношение количества возвратов в срок к общему количеству клиентов в гуппе
pivot_family_dept['total_percent'] = round((pivot_family_dept.total_clients-pivot_family_dept.total_dept) \
                                             / pivot_family_dept.total_clients * 100, 2)
pivot_family_dept

,family_status_id,family_status,total_clients,total_dept,total_percent
0,0,женат / замужем,12344,931,92.46
1,1,гражданский брак,4163,388,90.68
2,2,вдовец / вдова,959,63,93.43
3,3,в разводе,1195,85,92.89
4,4,Не женат / не замужем,2810,274,90.25


In [27]:
# -3-
#создадим сводную таблицу pivot_total_income_dept для анализа зависимости между уровнем дохода и возвратом кредита в срок
pivot_total_income_dept = df.pivot_table(index=['total_income_category'], values='debt', aggfunc=['count','sum'])
# переименуем колонки сводной таблицы
pivot_total_income_dept.columns = ['_'.join(a) for a in pivot_total_income_dept.columns]
pivot_total_income_dept = pivot_total_income_dept.reset_index()\
.rename(columns={'count_debt':'total_clients', 'sum_debt':'total_dept'})
# для анализа добавим колонку total_percent - отношение количества возвратов в срок к общему количеству клиентов в гуппе
pivot_total_income_dept['total_percent'] = round((pivot_total_income_dept.total_clients-pivot_total_income_dept.total_dept) \
                                             / pivot_total_income_dept.total_clients * 100, 2)
pivot_total_income_dept

,total_income_category,total_clients,total_dept,total_percent
0,A | 1000001 и выше |,25,2,92.00
1,B | 200001–1000000 |,5041,356,92.94
2,C | 50001–200000 |,16033,1360,91.52
3,D | 30001–50000 |,350,21,94.00
4,E | 0–30000 |,22,2,90.91


In [28]:
# -4-
#создадим сводную таблицу pivot_purpose_dept для анализа зависимости между целями кредита и возвратом кредита в срок
pivot_purpose_dept = df.pivot_table(index=['purpose_category'], values='debt', aggfunc=['count','sum'])
# переименуем колонки сводной таблицы
pivot_purpose_dept.columns = ['_'.join(a) for a in pivot_purpose_dept.columns]
pivot_purpose_dept = pivot_purpose_dept.reset_index()\
.rename(columns={'count_debt':'total_clients', 'sum_debt':'total_dept'})
# для анализа добавим колонку total_percent - отношение количества возвратов в срок к общему количеству клиентов в гуппе
pivot_purpose_dept['total_percent'] = round((pivot_purpose_dept.total_clients-pivot_purpose_dept.total_dept) \
                                             / pivot_purpose_dept.total_clients * 100, 2)
pivot_purpose_dept

,purpose_category,total_clients,total_dept,total_percent
0,операции с автомобилем,4308,403,90.65
1,операции с недвижимостью,10814,782,92.77
2,получение образования,4014,370,90.78
3,проведение свадьбы,2335,186,92.03


# Выводы : 

1.  Для анализа зависимости между количеством детей в семье и возвратом кредита в срок была создана сводная таблица *pivot_children_dept*. Колонка < total_percent >  показывает отношение количества возвратов кредитов в срок к общему количеству клиентов. 

Можно сделать выводы : 
* все клиенты, у которых 5 детей вернули кредит в срок, таких клиентов в данных всего 9;
* у большинства клиентов нет детей, и они чаще остальных (за исключением 9 клиентов с 5 детьми, которые не учитываем) возвращают кредит в срок - в 92.46 % случаях;
* чаще всего задержка возврата кредита у клиентов с 1, 2 и 4 детьми в семье (в 90.73% случаях возврат кредита в срок).


In [32]:
pivot_children_dept

,children_count,total_clients,total_dept,total_percent
0,0,14107,1063,92.46
1,1,4856,445,90.84
2,2,2128,202,90.51
3,3,330,27,91.82
4,4,41,4,90.24
5,5,9,0,100.00


2.  Для анализа зависимости между семейным положением и возвратом кредита в срок была создана сводная таблица *pivot_family_dept*. Колонка < total_percent > показывает отношение количества возвратов кредитов в срок к общему количеству клиентов. 

Можно сделать выводы : 
* чаще остальных в срок возвращают кредит клиенты с семейным статусом "вдовец / вдова" - в 93.43% случаях;
* большинство клиентов имеют статус "женат / замужем" и возвращают кредит в срок в 92.46% случаях - что является выше среднего показателя среди всех клиентов (91.89%);
* самый низкий показатель процента возврата кредита в срок у клиентов "Не женат / не замужем" - 90.25%	

In [36]:
pivot_family_dept

,family_status_id,family_status,total_clients,total_dept,total_percent
0,0,женат / замужем,12344,931,92.46
1,1,гражданский брак,4163,388,90.68
2,2,вдовец / вдова,959,63,93.43
3,3,в разводе,1195,85,92.89
4,4,Не женат / не замужем,2810,274,90.25


3.  Для анализа зависимости между уровнем дохода и возвратом кредита в срок была создана сводная таблица *pivot_total_income_dept*. Колонка < total_percent > показывает отношение количества возвратов кредитов в срок к общему количеству клиентов. 

Можно сделать выводы : 
* чаще остальных в срок возвращают кредит клиенты в категории D (30001 – 50000 руб) - в 94% случаях;
* большинство клиентов в категории С (50001–200000 руб) и возвращают кредит в срок в 91.52% случаях - что является ниже среднего показателя среди всех клиентов (91.89%);
* самый низкий показатель процента возврата кредита в срок у клиентов в категории E (0–30000 руб) - 90.91%

In [43]:
pivot_total_income_dept

,total_income_category,total_clients,total_dept,total_percent
0,A | 1000001 и выше |,25,2,92.00
1,B | 200001–1000000 |,5041,356,92.94
2,C | 50001–200000 |,16033,1360,91.52
3,D | 30001–50000 |,350,21,94.00
4,E | 0–30000 |,22,2,90.91


3.  Для анализа зависимости между целями кредита и возвратом кредита в срок была создана сводная таблица *pivot_purpose_dept*. Колонка < total_percent > показывает отношение количества возвратов кредитов в срок к общему количеству клиентов. 

Можно сделать выводы : 
* большинство клиентов берут кредит для "операции с недвижимостью" и чаще остальных возвращают кредит в срок в 92.77%;
* самый низкий показатель процента возврата кредита в срок кредитов на "получение образования" и "операции с автомобилем" - 90.78% и 90.65% соответственно

In [49]:
pivot_purpose_dept

,purpose_category,total_clients,total_dept,total_percent
0,операции с автомобилем,4308,403,90.65
1,операции с недвижимостью,10814,782,92.77
2,получение образования,4014,370,90.78
3,проведение свадьбы,2335,186,92.03


**Изходя из сделанных выводов можем получить модель идеального и нежелательного клиентов :**

* **модель идельного клиента** -  клиент с 5 детьми, семейный статус - вдовец / вдова, с потребностью в кредите в пределах категории D (30 001 – 50 000 руб) для операции с недвижимостью;
* **модель нежелательного клиента** - клиент с 1, 2 или 4 детьми в семье, семейный статус - "Не женат / не замужем, с потребностью в кредите в пределах категории E ( до 30 000 руб) для "получение образования" или "операции с автомобилем".